In [ ]:
# @title Enter the `PyRosettaCluster` output directory
from pathlib import Path

# @markdown ##### E.g., enter `/content/data`. If the path begins with `/content/drive/MyDrive`, you will be asked to mount Google Drive
output_path = "/content/drive/MyDrive/pyrosettacluster-examples_example-2" # @param {type:"string"}
output_path = Path(output_path)

if str(output_path).startswith("/content/drive/MyDrive"):
    from google.colab import drive
    drive.mount("/content/drive")


In [ ]:
# @title Create the original `pixi` project
import os
import shutil
from pathlib import Path

# Install pixi
if not shutil.which("pixi"):
    !curl -fsSL https://pixi.sh/install.sh | sh
    os.environ["PATH"] = f"{os.getenv('PATH')}{os.pathsep}/root/.pixi/bin"
!pixi --version

# Clone GitHub repository
repo_path = Path.cwd() / "pyrosettacluster-examples"
if not repo_path.is_dir():
    !git clone https://github.com/klimaj/pyrosettacluster-examples.git {repo_path}

# Setup pixi project
env_file = Path.cwd() / ".env"
env_file.write_text(
    "\n".join(
        [
            "PDB_MIRROR_PATH=",
            "CCD_MIRROR_PATH=",
            "LOCAL_MSA_DIRS=",
            "HBPLUS_PATH=",
            "X3DNA_PATH=",
            "DSSP_PATH=",
            "HHFILTER_PATH=",
            "MMSEQS2_PATH=",
            "COLABFOLD_LOCAL_DB_PATH_GPU=",
            "COLABFOLD_LOCAL_DB_PATH_CPU=",
            "COLABFOLD_NET_DB_PATH_GPU=",
            "COLABFOLD_NET_DB_PATH_CPU=",
            "FOUNDRY_CHECKPOINT_DIRS=",
        ]
    )
)
manifest_path = repo_path / "envs" / "pixi-2" / "pixi.toml"
if (manifest_path.parent / "pixi.lock").exists():
    !pixi install --manifest-path {manifest_path} --frozen --no-progress
else:
    !pixi install --manifest-path {manifest_path} --no-progress

In [ ]:
# @title Run the `PyRosettaCluster` simulation
from torch.cuda import is_available

scratch_dir = Path("/content/scratch")
gpu_flag = "--gpu" if is_available() else "--no-gpu"
!export $(xargs < {env_file}) && \
    cd {repo_path} && \
    pixi run --manifest-path {manifest_path} \
    python -m src.runners.example_2 {gpu_flag} \
    --output_path {output_path} \
    --scratch_dir {scratch_dir} \
    --num_tasks 1

In [ ]:
scorefile = output_path / "scores.bz2"
!cd {repo_path} && \
    pixi run --manifest-path {manifest_path} \
    python -m src.plotting.analyze_example_2 \
    --original_scorefile {scorefile}